# Lab 4

The security project revisited

In [ ]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import Markdown
load_dotenv(override=True)

semgrep_app_token = os.getenv("SEMGREP_APP_TOKEN")

In [ ]:
semgrep_params = {
      "command": "uvx",
      "args": ["semgrep-mcp"],
      "env": {
        "SEMGREP_APP_TOKEN": semgrep_app_token
      }
    }

In [ ]:
async with MCPServerStdio(params=semgrep_params, client_session_timeout_seconds=30) as semgrep:
    semgrep_tools = await semgrep.session.list_tools()

semgrep_tools.tools

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30,) as filesystem:
    file_tools = await filesystem.session.list_tools()

file_tools.tools

In [ ]:
instructions = """
You are a cybersecurity researcher. You are given the name of a python file to analyze. You have tools to use Semgrep to scan the code.
You read the file, you use the semgrep_scan tool to analyze the code.
Reply with the results of the analysis.
"""

with trace("Security Researcher"):
    async with MCPServerStdio(params=semgrep_params, client_session_timeout_seconds=120) as semgrep:
        async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
            agent = Agent(name="Security Researcher", instructions=instructions, model="gpt-4.1", mcp_servers=[semgrep, filesystem], mcp_config={"allowed_tools": ["semgrep_scan", "read_file"]})
            result = await Runner.run(agent, input="Please scan the file airline.py in the sandbox directory using the semgrep_scan tool and return the results")
            report = result.final_output

display(Markdown(report))